In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
#main idea to predict a customer's total amount expenditure(continuous money value)
data = spark.read.options(header=True, inferSchema=True).csv('/FileStore/tables/ecom.csv')

In [0]:
data.show(n=5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [0]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
data.head(1)[0]

Out[12]: Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [0]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [0]:
#set up the data frame
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

Out[15]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
#idea -> inputcol -> all the features presented in the last will be converted into a single vector

assembler = VectorAssembler(inputCols = ['Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                         'Length of Membership',
                                         'Yearly Amount Spent'],
                           outputCol='features' )

In [0]:
output = assembler.transform(data)

In [0]:
output.show(n=2)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|   Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|   Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+----

In [0]:
output.head(1)

Out[22]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [0]:
output.head(1)[-1][-1] #->features 

Out[24]: DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511])

In [0]:
#this is how spark expects data frame to be input for modeling
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show(n=5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [0]:
#splitting the data into train test 
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean|   498.475146633243|
| stddev|  77.77400652839094|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean|  501.3480631584063|
| stddev|  83.17195598516197|
|    min|   266.086340948469|
|    max|  700.9170916173961|
+-------+-------------------+



In [0]:
#creating LR model
lr = LinearRegression(labelCol='Yearly Amount Spent', featuresCol='features')

In [0]:
#fitting the data
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)
test_results.residuals.show(n=5)

+--------------------+
|           residuals|
+--------------------+
|7.958078640513122...|
|2.103206497849896...|
|5.684341886080801...|
|-2.61479726759716...|
|-2.27373675443232...|
+--------------------+
only showing top 5 rows



In [0]:
#difference predicted value and actual value and take square root
test_results.rootMeanSquaredError

Out[38]: 1.0987254845080351e-12

In [0]:
test_results.r2

Out[39]: 1.0

In [0]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

